# Part 1

### Import the needed libraries

In [2]:
conda install lxml;

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB
    certifi-2019.6.16          |           py36_0         154 KB
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    ------------------------------------------------------------
                                           Total:         4.8 MB

The following packages will be UPDATED:

    lxml:    4.2.5-py37hefd8a0e_0             --> 4.3.0-py36hefd8a0e_0
    openssl: 1.0.2r-h14c3975_0    conda-forge --> 1.0.2s-h7b6447c_0   

The following pac

In [3]:
import pandas as pd, numpy as np


In [50]:
conda install -c conda-forge geopy --yes;

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


### Read the webpage and find the table

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
toronto = tables[0]


### Replacing words "Not assigned" and delete such rows in the Borough column (we should ignore it).
### In the third row of code below I replace Not assigned Neighbourhood with the name from the Borough column.

In [6]:
#replace not assigned with Nan
toronto=toronto.replace('Not assigned',np.nan)
#drop nan in Borugh column inplace
toronto.dropna(axis=0,how='any',subset=['Borough'],inplace=True)
#data in Neighbourhood column should be replaced with Borough values
toronto['Neighbourhood'].replace(np.nan,toronto['Borough'],inplace=True)
#now you can see the head of my toronto table
toronto.head().sort_values('Borough')

,Postcode,Borough,Neighbourhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
6,M6A,North York,Lawrence Heights


### Here I prepare some values to start work with columns.
#### I need for values from Postcode column without duplicates.

In [7]:
# obtain data from Postcode column without duplicates; it will be 'plist'
plist=toronto.Postcode.drop_duplicates().to_frame()
plist.head()

,Postcode
2,M3A
3,M4A
4,M5A
6,M6A
8,M7A


In [8]:
# I named iterater for plist as plist_i and set it to zero
plist_i=0
# in while-loop I want to find and join all Neighbourhoods for each Postcode
# so, while plist_i is not equal to the length of plist...
while plist_i<len(plist):
# I will use adding (it is what I will add to Neighbourhoods)
    adding=''
# plist_x is the value of Postcode column in plist
    plist_x=plist.iloc[plist_i,0]
# with loc I can find Neighbourhoods which corresponds to Postcode values
    x=toronto.loc[(toronto['Postcode']==plist_x),'Neighbourhood']
# they should be joined into one line
    adding=', '.join(x)
# and all cells in Toronto table should be set to such line
    toronto.loc[(toronto['Postcode']==plist_x),'Neighbourhood']=adding
# then it should be repeated with all other Postcode values
    plist_i=plist_i+1
# to escape duplicates
toronto=toronto.drop_duplicates()
# now you can see shape and head of Toronto Table
print('Shape of Toronto Table is ', toronto.shape)
toronto.reset_index(drop=True,inplace=True)
toronto.head(10)

Shape of Toronto Table is  (103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"



# Part 2
#### As geocoder was not available, I could only use the csv file

In [9]:
gscoord = pd.read_csv('gscoord.csv')
gscoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Initially, I created two empty column, and then I started to fill
#### them with the values (Latitude and Longitude) from gscoord dataframe.

In [10]:
i=0
toronto['Latitude']=' '
toronto['Longitude']=' '
while i<len(toronto):
    x=toronto.iloc[i,0]
    toronto.iloc[i,3]=gscoord.loc[gscoord['Postal Code']==x,'Latitude'].values[0]
    toronto.iloc[i,4]=gscoord.loc[gscoord['Postal Code']==x,'Longitude'].values[0]
    i=i+1
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895


#### In the next step, I imported folium library,
#### chose to work with data about Toronto, Ontario
#### and added data about this region to the map.

In [11]:
# map rendering library
import folium
from geopy.geocoders import Nominatim
# I chose Toronto, Ontario 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Here I used my credentials from Foursquare service

In [ ]:
CLIENT_ID = 'deleted' # your Foursquare ID
CLIENT_SECRET = 'deleted' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### At the next step I dropped duplicates from toronto dataframe
#### and saved new dataframe as toronto_exploring

In [13]:
torontod = toronto.loc[toronto['Borough'].str.contains('Toronto'),'Borough'].drop_duplicates()
toronto_exploring=toronto.loc[toronto.Borough.isin(torontod),:]
toronto_exploring.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3789
15,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
19,M4E,East Toronto,The Beaches,43.6764,-79.293
20,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733


#### then I obrained the coordinates of Toronto, Ontario

In [14]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
map_toronto_exploring = folium.Map(location=[latitude, longitude], zoom_start=11)

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### ... and added coordinates from toronto_exporing to the map in the view of popup labels.

In [15]:
# add markers to map
for lat, lng, borough, Neighbourhood in zip(toronto_exploring['Latitude'], toronto_exploring['Longitude'], toronto_exploring['Borough'], toronto_exploring['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto_exploring)  
    
map_toronto_exploring

#### The next task for me was to obtain data from Foursquare about Toronto, Ontario and transform them to the dataframe.
#### I have used the way we learnt in Labs while learning this course.

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    #limit for length of list
    LIMIT=100 
    #create an empty list
    venues_list=[]
    #for every note from our dataframe function will...
    for name, lat, lng in zip(names, latitudes, longitudes):
        # ...create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # ...make the GET request with the previous created url and get json-formatted text
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # to return only relevant information for each nearby venue we should choose district name,
        # coordinates, venue_name, venue coordinates and categories
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
# now we can store that in dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#and rename columns    
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### This cell only provide data for the function that allows me to attain that dataframe.

In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
toronto_venues = getNearbyVenues(names=toronto_exploring['Neighbourhood'],
                                   latitudes=toronto_exploring['Latitude'],
                                   longitudes=toronto_exploring['Longitude']
                                  )

toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Here you can see the shape of my dataframe.

In [18]:
print('The shape of Toronto Dataframe is', toronto_venues.shape)

The shape of Toronto Dataframe is (1704, 7)


#### The next task I did, was sorting and screening the most frequent venue category in Toronto.

In [19]:
toronto_venues.groupby('Neighbourhood',as_index=False).count().sort_values('Venue Category', ascending=0).head(10)[['Neighbourhood','Venue Category']]

,Neighbourhood,Venue Category
0,"Adelaide, King, Richmond",100
7,"Chinatown, Grange Park, Kensington Market",100
31,St. James Town,100
30,"Ryerson, Garden District",100
16,"First Canadian Place, Underground city",100
14,"Design Exchange, Toronto Dominion Centre",100
10,"Commerce Court, Victoria Hotel",100
19,"Harbourfront East, Toronto Islands, Union Station",100
32,Stn A PO Boxes 25 The Esplanade,94
6,Central Bay Street,84


#### I chose the first 20 from them I saw how many venues in each of them.

In [20]:
x=toronto_venues.groupby('Venue Category',as_index=False).count().sort_values('Venue', ascending=0).head(20)[['Venue Category','Venue']]
x

,Venue Category,Venue
55,Coffee Shop,143
44,Café,88
191,Restaurant,52
137,Italian Restaurant,47
19,Bakery,40
128,Hotel,39
21,Bar,38
179,Pizza Place,33
175,Park,31
138,Japanese Restaurant,26


#### In the several next cells you can see some additional facts about them

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 241 uniques categories.


In [22]:
print('There are {} uniques Venues'.format(x['Venue'].unique().sum()), 'in top 20 Venue Categories.')

There are 672 uniques Venues in top 20 Venue Categories.


#### If I decided to seek for a place for living there, I would like to live near Gym, Park, Bakery, Fish Market, Grocery or Store.
#### Actually, I have tried to perform seeking with Excel, but it takes me for some more time, than just using python and clustering...

In [23]:
my_list=['Gym','Park','Bakery','Fish Market','Grocery','Store',\
         'Department Store','Miscellaneous Shop','Shopping Mall']
x=toronto_venues.loc[toronto_venues['Venue Category'].isin(my_list),'Venue']
print('There are {} unique Venues'.format(x.count()),'in my favourite list')

There are 116 unique Venues in my favourite list


#### so, there are 116 unique Venues from 238 unique categories. I would like to create some auto method to choose the best place for living.

#### I can easily learn if there is a district that corresponds to my wishes and also has metro station, train station or airport.
#### But there are none of such... (see below)

In [24]:
y=['Metro Station']
my_goal=toronto_venues.loc[\
                           (toronto_venues['Venue Category'].isin(my_list))&(toronto_venues['Venue Category'].isin(y))\
                           ,'Neighbourhood']
my_goal.count()

0

In [25]:
y=['Train Station']
my_goal=toronto_venues.loc[\
                           (toronto_venues['Venue Category'].isin(my_list))&(toronto_venues['Venue Category'].isin(y))\
                           ,'Neighbourhood']
my_goal.count()

0

In [26]:
y=['Airport Terminal', 'Airport']
my_goal=toronto_venues.loc[\
                           (toronto_venues['Venue Category'].isin(my_list))&(toronto_venues['Venue Category'].isin(y))\
                           ,'Neighbourhood']
my_goal.count()

0

In [27]:
y=['Theater','Event Space','Plaza','Movie Theater','Music Venue','College Rec Center','Lake','Video Game Store','Jazz Club','Recording Studio']
my_goal=toronto_venues.loc[\
                           (toronto_venues['Venue Category'].isin(my_list))&(toronto_venues['Venue Category'].isin(y))\
                           ,'Neighbourhood']
my_goal.count()

0

In [28]:
my_goal=toronto_venues.loc[\
                           (toronto_venues['Venue Category'].isin(my_list))\
                           ,['Neighbourhood','Venue']].groupby('Neighbourhood',as_index=False).count().sort_values('Venue',ascending=0)
my_goal.head()

,Neighbourhood,Venue
9,"Commerce Court, Victoria Hotel",9
24,St. James Town,8
0,"Adelaide, King, Richmond",7
25,Stn A PO Boxes 25 The Esplanade,7
6,"Chinatown, Grange Park, Kensington Market",7


#### As you can see, due to this table, I can opt for Commerce Court, Victoria Hotel...
##### more likely than other neighborhoods.
#### But what if there are some other types of venues which are not in my interests?
#### For instance, I do not like to live near restaurants, clubs, business centers and so on.

#### So, I can consider Neighborhoods that corresponds to my initial wishes as my preferable regions for residency.

In [29]:
print ('There are ', my_goal.count()[1], 'nerghborhood that correspond to my wishes.')

There are  30 nerghborhood that correspond to my wishes.


#### Since I have not in mind how I can distinguish neighborhoods of my wishes and neighborhoods
#### which are not appropriate for me, I should start to use clustering to attain my goal.

#### Now I can perform one hot encoding as it was described in this course
#### I need to do that to use categorical values in my investigation.
#### As the result, you can see what kind of Venues there are in each neighborhood.

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.tail()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
1699,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1700,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1701,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1702,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1703,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Here I just want to know what is frequency of each kind of venues.

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().round(4).reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.03,0.0,...,0.0,0.0,0.0,0.0,0.0100,0.0,0.0,0.01,0.0,0.0000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0179,0.0,0.0,0.00,0.0,0.0000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0,0.0435
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0,0.0556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.00,0.0,0.0000


In [32]:
toronto_grouped.shape

(38, 242)

#### Now I want to obtain information about 5 the most common venues in each neighborhoods.

In [33]:
# firstly, I grouped my dataframe by neighborhoods
toronto_grouped1 = toronto_onehot.groupby('Neighbourhood').mean().round(4)
# then, I created the initial list for new dataframe (list will be changed)
toronto_v_top=pd.DataFrame(['1','2','3','4','5','6','7','8','9','10'])
# then, I created the initial list for header for new dataframe
toronto_v_columns=[]
k=0
while k<len(toronto_grouped1):
    # firstly, I transposed my dataframe, and sorted it from high to low
    n_toronto_grouped1=pd.DataFrame(toronto_grouped1.iloc[k,:].transpose().sort_values(ascending=False))
    #got index from transposed dataframe (I got the first 5 of them to obtain 5 the most common Venue)
    toronto_v_top[k]=pd.DataFrame(n_toronto_grouped1.head(10)).index
    #and added columns name (which are the name of neighborhoods) to the list of future columns
    toronto_v_columns.append(pd.DataFrame(n_toronto_grouped1.head(10)).columns.values.tolist())
    k=k+1
    


In [34]:
#after perfoming tasks from the previous cell, I can create the columns,
toronto_v_columns=pd.DataFrame(toronto_v_columns)
# and change the name of columns in created dataframe
toronto_v_top.columns=toronto_v_columns[0]
#and perform transpositioin again
toronto_v_top=toronto_v_top.T

#Again, I need the names of new dataframe columns,
toronto_v_top.columns=['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue',\
                      '6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']
#reset index,
toronto_v_top.reset_index(level=0, inplace=True)
#rename the first column
toronto_v_top.rename(columns={0:'Neighbourhood'},inplace=True)
#and check my result
toronto_v_top.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Hotel,Asian Restaurant,Restaurant,Breakfast Spot,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Yoga Studio,Burrito Place,Sandwich Place,Restaurant,Caribbean Restaurant,Climbing Gym,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Coffee Shop,Sculpture Garden,Bar,Boat or Ferry,Airport Gate


#### I can use the method from our Lab to create the Cluster Labels for Venues in my dataframe

In [35]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
# add clustering labels

toronto_v_top.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_v_top.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged=toronto_merged.dropna(axis=0,inplace=False,how='any',subset=['Cluster Labels']).sort_values('Cluster Labels',ascending=False)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.6969,-79.4113,6.0,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
61,M4N,Central Toronto,Lawrence Park,43.728,-79.3888,5.0,Dim Sum Restaurant,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.6896,-79.3832,4.0,Restaurant,Playground,Tennis Court,Trail,Summer Camp,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
62,M5N,Central Toronto,Roselawn,43.7117,-79.4169,3.0,Garden,Home Service,Pool,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
19,M4E,East Toronto,The Beaches,43.6764,-79.293,2.0,Health Food Store,Neighborhood,Trail,Pub,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop


In [37]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
 #       fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels']==5.0,\
                   ['Cluster Labels','Neighbourhood','1st Most Common Venue','2nd Most Common Venue',\
                    '3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue', '6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,5.0,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [40]:
toronto_merged.shape

(38, 16)

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0.0,\
                   ['Cluster Labels','Neighbourhood','1st Most Common Venue','2nd Most Common Venue',\
                    '3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue', '6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']]\
.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,0.0,"Runnymede, Swansea",Pizza Place,Café,Sushi Restaurant,Coffee Shop,Italian Restaurant,Latin American Restaurant,Indie Movie Theater,Bar,Fish & Chips Shop,Spa
73,0.0,North Toronto West,Sporting Goods Shop,Clothing Store,Coffee Shop,Restaurant,Rental Car Location,Miscellaneous Shop,Salon / Barbershop,Diner,Mexican Restaurant,Chinese Restaurant
74,0.0,"The Annex, North Midtown, Yorkville",Coffee Shop,Sandwich Place,Café,Pizza Place,Pharmacy,Burger Joint,Jewish Restaurant,Flower Shop,Indian Restaurant,Pub
75,0.0,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Movie Theater,Dessert Shop,Eastern European Restaurant,Dog Run,Bar,Bank,Italian Restaurant,Restaurant
79,0.0,Davisville,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Thai Restaurant,Flower Shop,Pharmacy


In [42]:
toronto_merged.loc[(toronto_merged['Cluster Labels']!=0.0),\
                   ['Cluster Labels','Neighbourhood','1st Most Common Venue','2nd Most Common Venue',\
                    '3rd Most Common Venue','4th Most Common Venue','5th Most Common Venue', '6th Most Common Venue','7th Most Common Venue','8th Most Common Venue','9th Most Common Venue','10th Most Common Venue']]

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,6.0,"Forest Hill North, Forest Hill West",Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
61,5.0,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
83,4.0,"Moore Park, Summerhill East",Restaurant,Playground,Tennis Court,Trail,Summer Camp,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
62,3.0,Roselawn,Garden,Home Service,Pool,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
19,2.0,The Beaches,Health Food Store,Neighborhood,Trail,Pub,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop
91,1.0,Rosedale,Park,Playground,Trail,Building,Yoga Studio,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant


#### I should remember about my goal (i.e. preferable regions):

In [43]:
my_goal

,Neighbourhood,Venue
9,"Commerce Court, Victoria Hotel",9
24,St. James Town,8
0,"Adelaide, King, Richmond",7
25,Stn A PO Boxes 25 The Esplanade,7
6,"Chinatown, Grange Park, Kensington Market",7
12,"Design Exchange, Toronto Dominion Centre",7
14,"First Canadian Place, Underground city",6
17,"Harbourfront, Regent Park",6
16,"Harbourfront East, Toronto Islands, Union Station",6
23,"Ryerson, Garden District",5


#### Finally, you can see that in the case I used only information about what I wish to have in my district, I should choose Commerce Court or Victoria Hotel.
#### But with using clustering I can see that they are not the best choice for me: these districts are more entertaining than, for instance Lawrence Park or Moore Park.
#### But the best options for me are 1) Lawrence Park, 2) Moore Park and Summerhill East and 3) Rosedale
#### because all of them do not contradict to my_goal list and have enough diversity in venue species which can be useful for me.
